In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# Load in Data

We'll load the Pride and Prejudice text file, an 1813 romantic novel of manners written by Jane Austen, and convert it into integers for our network to use.

In [10]:
# open text file and read in data as `text`
with open('data/pride-and-prejudice.txt', 'r') as f:
    text = f.read()
    text = ''.join([i for i in text if not i.isdigit()])

Let's check out the first 100 characters, make sure everything is peachy.

In [11]:
text[:100]

'\n      It is a truth universally acknowledged, that a single man in\n      possession of a good fortu'

# Tokenization
We'll create a dictionary out of all the characters that we have in the sentences and map them to an integer. This will allow us to convert our input characters to their respective integers (char2int) and vice versa (int2char).

In [12]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

First few encoded characters

In [17]:
encoded[:100]

array([15, 57, 57, 57, 57, 57, 57, 11, 68, 57,  9, 13, 57, 34, 57, 68, 49,
        3, 68, 67, 57,  3,  4,  9, 61, 69, 49, 13, 34, 47, 47, 40, 57, 34,
       43, 65,  4, 53, 56, 47, 69,  0, 18, 69,  0, 36, 57, 68, 67, 34, 68,
       57, 34, 57, 13,  9,  4, 18, 47, 69, 57,  2, 34,  4, 57,  9,  4, 15,
       57, 57, 57, 57, 57, 57, 19, 53, 13, 13, 69, 13, 13,  9, 53,  4, 57,
       53, 35, 57, 34, 57, 18, 53, 53,  0, 57, 35, 53, 49, 68,  3])

# Pre-processing the data
LSTM expects an input that is one-hot encoded meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that.

In [26]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [27]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


# Creating batches

In [28]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# Test Your Implementation
Now I'll make some data sets and we can check out what's going on as we batch data. 
Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [30]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[15 57 57 57 57 57 57 11 68 57]
 [57 18  9 49 47 13 57 53 35 57]
 [57 57 35  9  4  0 57 68 67 34]
 [67  9 13 57 13 69 43 53  4  0]
 [ 9 68 57 53 35 57 40 53  3 49]
 [ 4 69 57 19 69 49 13 53  4 36]
 [ 4 57 35  9  4  0  9  4 18 57]
 [57 67 34 61 69 15 57 57 57 57]]

y
 [[57 57 57 57 57 57 11 68 57  9]
 [18  9 49 47 13 57 53 35 57 49]
 [57 35  9  4  0 57 68 67 34 68]
 [ 9 13 57 13 69 43 53  4  0 57]
 [68 57 53 35 57 40 53  3 49 57]
 [69 57 19 69 49 13 53  4 36 38]
 [57 35  9  4  0  9  4 18 57 53]
 [67 34 61 69 15 57 57 57 57 57]]


Before we start building the model, let's use a built-in feature in PyTorch to check the device we're running on (CPU or GPU). This implementation will not require GPU as the training is really simple. However, as you progress on to large datasets and models with millions of trainable parameters, using the GPU will be very important to speed up your training.

In [32]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [33]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

### Creating a training method
The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [43]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    valid_loss_min = np.Inf 
    
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                
                if np.mean(val_losses) < valid_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, np.mean(val_losses)))
                    # change the name, for saving multiple files
                    model_name = 'rnn_20_epoch.net'

                    checkpoint = {'n_hidden': net.n_hidden,
                                  'n_layers': net.n_layers,
                                  'state_dict': net.state_dict(),
                                  'tokens': net.chars}

                    with open(model_name, 'wb') as f:
                        torch.save(checkpoint, f)
                        valid_loss_min = np.mean(val_losses)
                

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [46]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(70, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=70, bias=True)
)


In [47]:
batch_size = 128
seq_length = 100
n_epochs = 2 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/2... Step: 10... Loss: 3.1170... Val Loss: 3.1271
Validation loss decreased (inf --> 3.127068). Saving model...


KeyboardInterrupt: 

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [48]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [49]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [52]:
print(sample(net, 1000, prime='The', top_k=5))

Thete teaa e oe  aet    teetea  e  ee oo  ttoe a ea eoo oto   oa toooo t t ee ee oo aee ta o e  e ao e o ae   e  ta   aaee  e  o  e t   e o a e ttta  e a   at ettttt te o   a  teeeetoae  t oat  t e teoo e oaeeote    te ot o  ta aa       o    o  oo ea oee  t   o ttaoee    o  ao  ete  a    oa   o  e ee  tat eaoeo tt tea     ee e   te ato o ea o    o  aee tttt t eta t oee  eaaa  o  oe   e t e   aoe  tet a  aa      o   e aa o e     a to  ee oea tt   oot ee  ae aot   et   t e   eto e     te et eee     t   eo   a o  t   ea o   oa aet  t  a t    t e    t   et ee eo aate ee t     et   t  oe t    ot   ae      tooa  eaeae  eeo a eot    toeoe  ea   a a o     ett  e   tate   o  aoe   ae e     et tee t  a e     o  ea et  e eto  e  a t tt    eae   a t a  e a  a  o    ae oa      e  eteo  et  e  t  totao e t  e     aeeaa     eee t    o eao   toea  a  o   te  te  oa t a  tet   e  oeo t   eet   tea  oo t  oa o  tte ee       e    e t o e o a     et   e   t  ee oet e  t eeee   oe e  o     tt  te e  o  ee 

## Loading a checkpoint

In [53]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [54]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin saidniieit  eeeeee t a enantnteeeatet t aten  annnen e neettea eaeeeeee    ttat an  t ttnae na t etn nnnnn tenete aaeatt tea atttataetneneeeenteaaneata eatnt eenn n teantaaaeee ta naaaate eananne taa aeenett ntnat e enenta t naentnenet aaaat e aenneat enennn e etaen ttt e  e  t e an etataaentt eena  en  e eattneantn ntenaa at e aeeetntenen t eeaatnt e ne n eentennaaene tee tnaat   etetetatte en e e teeaa enaee eee  e neanean aaa tee ea en ee  neneneeeat taatttennetea etnneeetett eeaaaat ttan ee nneet e tnntnaeaneetea etaann etan eaaaeetatte ee  eteaeetaenne aeaeet tntn  a net nen ee e  tneeaaeanntne ea tnaetenntnee  eeetttt nee eteeet ene net n eaa eaeeet aeeneaa ataeet e  ana ea nnne  aeeaet ee neae een eet aetnn  nnta e nttneeeenetnee  te nenet ataaeen neant eeeeetnnnnnen atetnea neneeae   aea etteaneeeneaett nenteta eeteennnen atene ane ee aeete eta neaeee ttne at aa  atnetettnenetaee tan atn ttaaen a aeate ene aeat nn teaaeaeaee  tateeet etneante ea nttttatteat tnenen  ta